# Importing API Key

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

if not HUGGINGFACE_API_KEY:
    raise ValueError("HUGGINGFACE_API_KEY not found in .env file")

In [4]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=HUGGINGFACE_API_KEY)
messages = [
	{ "role": "user", "content": "Tell me a story" }
]

completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct", 
	messages=messages, 
	temperature=0.5,
	max_tokens=2048,
	top_p=0.7
)

print(completion.choices[0].message)

f:\Programs\anaconda3\envs\torchdl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatCompletionOutputMessage(role='assistant', content='Certainly! Here’s a short story for you:\n\n### The Lighthouse Keeper and the Lost Sailor\n\nIn a small coastal village, there lived an old lighthouse keeper named Thomas. Every night, he would climb the winding stairs of the lighthouse to light the beacon, ensuring that the ships passing by would safely navigate the treacherous waters. Thomas had been the keeper for over 40 years, and he took his job very seriously. The lighthouse was his life, and he knew every inch of it.\n\nOne stormy night, as the wind howled and the rain lashed against the lighthouse, Thomas noticed something unusual. A small boat was struggling against the waves, its lights flickering weakly. Thomas immediately knew that the boat was in grave danger. He quickly lit the beacon and increased its intensity, hoping to guide the boat to safety.\n\nFor what felt like hours, Thomas watched the boat as it battled the storm. Just when he thought all hope was lost, th

In [5]:
completion.choices[0].message['content']

'Certainly! Here’s a short story for you:\n\n### The Lighthouse Keeper and the Lost Sailor\n\nIn a small coastal village, there lived an old lighthouse keeper named Thomas. Every night, he would climb the winding stairs of the lighthouse to light the beacon, ensuring that the ships passing by would safely navigate the treacherous waters. Thomas had been the keeper for over 40 years, and he took his job very seriously. The lighthouse was his life, and he knew every inch of it.\n\nOne stormy night, as the wind howled and the rain lashed against the lighthouse, Thomas noticed something unusual. A small boat was struggling against the waves, its lights flickering weakly. Thomas immediately knew that the boat was in grave danger. He quickly lit the beacon and increased its intensity, hoping to guide the boat to safety.\n\nFor what felt like hours, Thomas watched the boat as it battled the storm. Just when he thought all hope was lost, the boat managed to find a small cove near the village. 

# Creating a Pipeline For Chatbot

In [13]:
from typing import List, Dict
class ChatBot:
    def __init__(self, api_token: str):
        self.client = InferenceClient(api_key=api_token)
        self.model = "Qwen/Qwen2.5-72B-Instruct"

    def generate_response(self, context: str, chat_history: List[Dict[str, str]]) -> str:
        """Generate response using context and chat history"""
        
        system_msg = "You are a helpful AI assistant that provides clear and concise information based on the given context and chat history."
        
        formatted_history = []
        for msg in chat_history:
            formatted_history.append({"role": msg["role"], "content": msg["content"]})
            
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": f"Context: {context}"},
        ]
        messages.extend(formatted_history)
        print(messages)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.9
        )
        
        return response.choices[0].message.content
    
    def query(self, query:str):
        messages = [
            { "role": "user", "content": query }
        ]

        completion = client.chat.completions.create(
            model=self.model, 
            messages=messages, 
            temperature=0.5,
            max_tokens=2048,
            top_p=0.7
        )

        return completion.choices[0].message.content

# Testing it

In [15]:
chatbot = ChatBot(api_token=HUGGINGFACE_API_KEY)
    
context = "Python is a high-level programming language."
history = [
    {"role": "user", "content": "What is Python?"},
    {"role": "assistant", "content": "Python is a programming language."},
    {"role": "user", "content": "What are its key features?"}
]

response = chatbot.generate_response(context, history)
print(response)

[{'role': 'system', 'content': 'You are a helpful AI assistant that provides clear and concise information based on the given context and chat history.'}, {'role': 'user', 'content': 'Context: Python is a high-level programming language.'}, {'role': 'user', 'content': 'What is Python?'}, {'role': 'assistant', 'content': 'Python is a programming language.'}, {'role': 'user', 'content': 'What are its key features?'}]
Python is known for several key features that make it a popular choice for a wide range of applications:

1. **Ease of Learning and Use**: Python has a simple, clear syntax that is easy to read and write, making it an excellent language for beginners.

2. **Interpreted Language**: Python is an interpreted language, which means that it is executed line by line, making debugging easier and more straightforward.

3. **High-Level Language**: Python abstracts many low-level details, allowing developers to focus on solving problems rather than managing system architecture and memo

# Testing Query function

In [17]:
chatbot.query("What is Python?")

"Python is a high-level, interpreted programming language created by Guido van Rossum and first released in 1991. It is designed to be easy to read and write, with a syntax that emphasizes code readability and reduces the cost of program maintenance. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming.\n\nKey features of Python include:\n\n1. **Easy to Learn and Use**: Python has a simple and straightforward syntax, making it an excellent choice for beginners and experienced developers alike.\n2. **Interpreted Language**: Python code is executed line by line, which makes debugging easier and allows for interactive testing and development.\n3. **Dynamically Typed**: You don't need to declare the type of a variable when you create one, and types are checked at runtime.\n4. **Extensive Standard Library**: Python comes with a large standard library that includes modules and packages for a wide range of tasks, from web developmen